"Coursera DeepLearning AI Deep Learning Specialization Sequence Models"

"Coursera DeepLearning AI NLP Specialization"

- Tensorflow NLP Tutorial, Tensorflow Hub NLP Models
- Huggingface
- AllenNLP
- Pytorch

# NLP Tasks

In this notebook, we will examine some of the currently popular NLP tasks. We use some of the popular open-sourced NLP frameworks/libraries to do that.

There will be no training (backpropagation) in this notebook. We are only interested in making predictions with models, to understand the nature of the tasks.

**Tasks to cover in this notebook:**

1. (extractive) Question Answering
2. Named Entity Recognition (also called token tagging, or, entity extraction)
3. Sentiment Analysis (a specific case of text classification)
4. Text Generation
5. Conversetional Agents

**Important eye-opener:** All of the tasks in this notebook could be modelled in another way than we do here. However, we tried to show you the most popular ways as of 2021 to perform these tasks. For example, sentiment analysis task could be modelled as a regression problem, rather than a multi-class classification problem.

## 1- Using AllenNLP

AllenNLP is an open source NLP framework formed by Allen Institute for AI, a private company & research institute. AI2 is founded by Paul Allen, one of the co-founders of Microsoft.

![image.png](attachment:84ffb7d1-ee1a-4ed7-b31f-0242d978f0df.png)

____

Codes below are obtained from AllenNLP demo site. There are more tasks there. We will examine only three of them.

- https://demo.allennlp.org

### a- Reading Comprehension (Question Answering)

Question answering is the task of extracting text spans from context paragraphs. While doing the extraction model also takes a question into account.

- Inputs: Context Paragraph (tokenized string), Question (tokenized string)
- Outputs: A text span (de-tokenized list of tokens), named as the "Answer"
- Popular Methods: BERT based QA models

![QA.jpg](attachment:e040814c-2db4-4b88-9a40-2370a67eaead.jpg)

In the example, we see a context paragraph, giving information on inzva. Then, we have a question, asking inzva's foundation date.

We basically ask the model this question: 

- **If we have N tokens in the context, what is the likelihood of $n^{\text{th}}$ token to be the start token of the answer. Likewise, what is the likelihood for each token to be the ending token of the answer?**
- By asking that, we find the most likely start and end positions for the answer span.
- Then we can get the answer span between the most likely start token and the most likely end token. 
- In the example, token "2017" is the most likely token to be the start token. It is also most likely to be the end token. Between the most likely start token and the most likely end token, the only word is "2017" so we take that as the answer.

___

We install allennlp_models to use the pretrained models in AllenNLP

In [ ]:
!pip install allennlp allennlp_models
!pip install --upgrade google-cloud-storage

Then we import the Predictor class, which is a wrapper for ML Model classes.

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.rc

We instantiate a Predictor instance which is a QA Model in our case.

In [ ]:
qa_model = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-elmo-model-2020.03.19.tar.gz")

We make the predictions with the two necessary parameters. Question and the context passage. The answer will be selected from inside the context passage.

In [38]:
results = qa_model.predict(
  passage="The Matrix is a 1999 science fiction action film written and directed by The Wachowskis, starring Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, and Joe Pantoliano.",
  question="What was the budget of The Matrix?")

/usr/local/lib/python3.7/dist-packages/allennlp_models/rc/models/utils.py:32: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  span_start_indices = best_spans // passage_length


We print three things:
- Our context (just to see it)
- The question (just to see it)
- The answer, which is in the key *best_span_str* .

In [46]:
results = qa_model.predict(
  passage="The Matrix is a 1999 science fiction action film written and directed by The Wachowskis, starring Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, and Joe Pantoliano.",
  question="When was the Matrix produced?")

/usr/local/lib/python3.7/dist-packages/allennlp_models/rc/models/utils.py:32: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  span_start_indices = best_spans // passage_length


In [47]:
for i,item in enumerate(zip(results["span_start_probs"],results["span_end_probs"])):
  print(item,results["passage_tokens"][i])

(0.0059886048547923565, 2.97971346299164e-05) The
(0.0001135591883212328, 0.0006942124455235898) Matrix
(5.7538458349881694e-05, 6.995798685238697e-06) is
(0.010700530372560024, 3.842319074465195e-06) a
(0.9811098575592041, 0.926461935043335) 1999
(0.0010354286059737206, 0.001707791117951274) science
(9.498111467109993e-05, 0.06849149614572525) fiction
(0.0007048737606965005, 0.0002805436379276216) action
(6.459171345341019e-06, 0.0015518845757469535) film
(3.282208854216151e-05, 3.1104493245948106e-05) written
(7.363278655247996e-06, 5.626451638818253e-06) and
(4.694305062002968e-06, 1.7014330296660773e-05) directed
(3.94298604078358e-06, 3.605018264352111e-06) by
(3.9204904169309884e-05, 1.9755302673729602e-06) The
(3.605255187721923e-05, 0.00020484124252106994) Wachowskis
(2.881515115404909e-07, 2.4242624931503087e-05) ,
(7.164513249335869e-07, 2.4717949145269813e-06) starring
(1.8673377780942246e-05, 2.75472120847553e-06) Keanu
(2.4421214561698434e-07, 6.814272637711838e-05) Reeves

In [45]:
for i,item in enumerate(zip(results["span_start_probs"],results["span_end_probs"])):
  print(item,results["passage_tokens"][i])

(0.3534693419933319, 0.012056135572493076) The
(0.0026501340325921774, 0.07330133765935898) Matrix
(0.0023868922144174576, 0.0028157872147858143) is
(0.05499861389398575, 0.0011885670246556401) a
(0.4507535994052887, 0.3042233884334564) 1999
(0.04138783738017082, 0.009345144033432007) science
(0.0032744486816227436, 0.25432443618774414) fiction
(0.02548595890402794, 0.003398345550522208) action
(0.0006183712393976748, 0.0387117974460125) film
(0.0015431317733600736, 0.0017001550877466798) written
(0.000730712607037276, 0.0003484560002107173) and
(0.00040637326310388744, 0.0013140913797542453) directed
(8.344931848114356e-05, 0.00019434303976595402) by
(0.007540808524936438, 8.336621976923198e-05) The
(0.007430798374116421, 0.026591921225190163) Wachowskis
(5.0442733481759205e-05, 0.002453346736729145) ,
(6.61697777104564e-05, 0.00025419832672923803) starring
(0.002071676542982459, 0.0003216151671949774) Keanu
(6.931848474778235e-05, 0.007053593639284372) Reeves
(9.893746209854726e-06, 

In [44]:
results.keys()

dict_keys(['passage_question_attention', 'span_start_logits', 'span_start_probs', 'span_end_logits', 'span_end_probs', 'best_span', 'best_span_str', 'question_tokens', 'passage_tokens', 'token_offsets'])

In [39]:
print(' '.join(results['passage_tokens']), ' '.join(results['question_tokens']), results['best_span_stys(r'], sep = '\n\n')

The Matrix is a 1999 science fiction action film written and directed by The Wachowskis , starring Keanu Reeves , Laurence Fishburne , Carrie - Anne Moss , Hugo Weaving , and Joe Pantoliano .

What was the budget of The Matrix ?

1999


### b- Named Entity Recognition (Tagging, Entity Extraction)


Entity recognition is the task of extracting particular spans from text, and sometimes classifying them to classes like *Person* , *Location* , *Orgazination* etc.

![NER.jpg](attachment:5be6d8ee-f937-437d-983d-7af7a2423618.jpg)

In the image, we see an input sentence with 4 input words. NER Model takes the input, and assigns an output class for each token. In this example, outputs are:
- inzva -> Organization
- is -> None
- in -> None
- Beykoz -> Location

Normally, tokens would be subwords, instead of words. In our example we consider words as tokens for simplicity.

To start getting predictions on NER, we import the tagging model from AllenNLP

In [ ]:
import allennlp_models.tagging

We again create a Predictor instance, which will be our model.

In [ ]:
ner_model = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-model-2020.02.10.tar.gz")

The model makes a prediction.

In [29]:
results = ner_model.predict(
  sentence="Google was founded on September 4, 1998, by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California.")

We examine the prediction, looking at its keys.

In [30]:
results.keys()

dict_keys(['logits', 'mask', 'tags', 'words'])

We see that "words" and "tags" keys might be relevant to print out each word with their found class. So we print those lists.

In [31]:
for word, tag in zip(results["words"], results["tags"]):
    print(f"{word}\t{tag}")

Google	U-ORG
was	O
founded	O
on	O
September	O
4	O
,	O
1998	O
,	O
by	O
Larry	B-PER
Page	L-PER
and	O
Sergey	B-PER
Brin	L-PER
while	O
they	O
were	O
Ph.D.	U-MISC
students	O
at	O
Stanford	B-ORG
University	L-ORG
in	O
California	U-LOC
.	O


### c- Sentiment Analysis

Sentiment Analysis is a text classification problem. One of the easiest ways to do it is to classify the text either as postive or negative.

![Sentiment Analysis(2).jpg](attachment:71209240-7a14-4ed3-97d4-8bc5ba5720f2.jpg)

In the example we see that the person thinks that the movie sucks (is bad). 

This is because the person states that if they said the movie *didn't* suck, it would be a *lie* (this is a double negation example).

We import the classification module since the way that we model sentiment analysis is a classification problem.

In [ ]:
import allennlp_models.classification

We start a predictor class witht the pretrained sentiment analysis model's path.

In [ ]:
sentiment_classifier = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/basic_stanford_sentiment_treebank-2020.06.09.tar.gz")

We write a negative review.

In [62]:
# Review of the movie "Joker" on Rotten Tomatoes
review = "Begenmedim ben bu filmi"

We forward our negative review to the model.

In [63]:
result = sentiment_classifier.predict(sentence=review)

We print "positive" if the model's confidence is higher on being positive than negative. 

Notice that AllenNLP models the sentiment analysis task in a more complex way than binary classification. 

They have multiple classes and also confidence values associated with each class.

In [64]:
print('Positive with confidence of', result['probs'][0]) if result['label']=='1' else print('Negative with confidence of', result['probs'][1])

Positive with confidence of 0.9953984618186951


## 2- Using Huggingface Transformers

Transformers is an open source NLP framework formed by Huggingface, a private company & community.

![image.png](attachment:d93e6329-539b-4fb4-a4e0-0c0da8e66c13.png)

Note: You can also check https://huggingface.co/spaces to get a wider sense on NLP (and also other machine learning) tasks. 

____

### Using "Pipeline" for Sentiment Analysis

Pipeline class is a wrapper in Huggingface, just like the Predictor in AllenNLP. 

It basically creates a more specialized class instance under the hood, based on the task name that you give. 

Sentiment Analysis is a text classification problem. One of the easiest ways to do it is to classify the text either as postive or negative.

![Sentiment Analysis(2).jpg](attachment:71209240-7a14-4ed3-97d4-8bc5ba5720f2.jpg)

In the example we see that the person thinks that the movie sucks (is bad). 

This is because the person states that if they said the movie *didn't* suck, it would be a *lie* (this is a double negation example).

In [ ]:
# Have either PyTorch >= 1.1 or TensorFlow >= 2.0. Then run the following command to install HuggingFace:
!pip install transformers

We start a Pipeline instance, particularly, a sentiment analysis pipeline. 

Right after we create it, we forward an input to it. The input is "we love you".

In [ ]:
from transformers import pipeline

print(pipeline('sentiment-analysis')('we love you'))

### Using "Pipeline" for Question Answering

Question answering is the task of extracting text spans from context paragraphs. While doing the extraction model also takes a question into account.

- Inputs: Context Paragraph (tokenized string), Question (tokenized string)
- Outputs: A text span (de-tokenized list of tokens), named as the "Answer"
- Popular Methods: BERT based QA models

![QA.jpg](attachment:e040814c-2db4-4b88-9a40-2370a67eaead.jpg)

In the example, we see a context paragraph, giving information on inzva. Then, we have a question, asking inzva's foundation date.

We basically ask the model this question: 

- **If we have N tokens in the context, what is the likelihood of $n^{\text{th}}$ token to be the start token of the answer. Likewise, what is the likelihood for each token to be the ending token of the answer?**
- By asking that, we find the most likely start and end positions for the answer span.
- Then we can get the answer span between the most likely start token and the most likely end token. 
- In the example, token "2017" is the most likely token to be the start token. It is also most likely to be the end token. Between the most likely start token and the most likely end token, the only word is "2017" so we take that as the answer.

___

We start a QA pipeline, and name it as "nlp". 

We also define one of our inputs (the context). It is an informative paragraph on one of the subtopics of Machine Learning. 

In [ ]:
nlp = pipeline("question-answering")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
"""

We define the other part of our input (the question). It is a question about one of the subtopics in Machine Learning.

We can ask multiple questions for the same context paragraph, and vice versa in certain cases. For that, we ask another question.

We print: 
- the answer span
- the score for the answer (similar to confidence, but not a probability value)
- the location (index) of the start of the answer
- the location of the end of the answer 

In [ ]:
result = nlp(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

result = nlp(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

### Text Generation with GPT-2

- Text Generation is essentially a language modelling task, however, lately it has started to become more.

- Some text generation models (like GPT-2) can be fine-tuned to perform a huge variety of NLP tasks (QA, classification etc.). 

- Some newer text generation models (like GPT-3 and GPT-J) can even zero-shot a wider variety of tasks.

![Untitled Diagram.jpg](attachment:0ae18278-ba30-4d2a-915a-6d9bb5d953b8.jpg)


In this example, the task solely Text Generation, similar to writing a story. We will start the story, and expect the model to write new lines. We will use GPT-2 due to practical limitations.

We import specific GPT model and tokenizer classes to form our predictive system, then create tokenizer and model instances.

In [ ]:
# For this example, tensorflow >= 2.1 should be installed on the system

import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

We tokenize the input sentence (the beginning of our story). You can examine the tokenized object if you want. 

In [ ]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

We generate the continuation of our story. Our arguments specify:
- max_length=50: do not generate more than 50 tokens
- num_beams=5: when you are thinking about the best possible stories, do not think about more than 5 stories at the same time.
- no_repeat_ngram_size=2: ?
- early_stopping=True: ?

We get the output (it is a list of tokens) then we join all of them to get one string.

In [ ]:
beam_output = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=False))

### Conversational Agent

Conversational Agents are multi-component systems that are engineered to act as Chatbots. They take action for various queries:

- Order a pizza.
- Call my friend Suzie.
- What's the weather like today?
- ...

In this example, we are trying out an end-to-end conversational agent, that is only designed for chitchat. 

It is a finetuned version of one of the GPT models, to do chitchat.

![Untitled Diagram(1).jpg](attachment:0d8f087b-8cae-4231-ae1a-b40d12c6de4f.jpg)

In [ ]:
# intent recognition (text classification)
# response generation

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

In [ ]:
# Does money buy happiness?
# what is the meaning of life ?
# who is the first president of the United States
# which one is bigger, sun or moon?
# what is the boiling point of water?
# who won the world cup in 2018 ?
# Nvidia's Titan RTX is really good .